# Joint conf-call report #3

In [1]:
import datetime
print(str(datetime.datetime.today()))

2018-05-17 11:07:32.458857


# Formalism adopted

The following are most common formalisms:
- uppercase bold letter like $\mathbf{A}$ indicates a matrix;
- lowercase bold letter like $\mathbf{x}$ indicates a column vector ($\mathbf{x}^T$ is a row vector);
- lowercase normal letter like $x$ indicates a mono-dimensional variable;
- usually subscript stands for _index access_, e.g. $x_i$ is the $i$-th component of $\mathbf{x}$;
- superscript within brackets stands for a sequence that embodies the history of a variable (or anything else), like $\pi^{(0)},\pi^{(1)},...,\pi^{(t-1)},\pi^{(t)}$;

Therefore:
- the training set is $\mathcal{X} = (\mathbf{X}, \mathbf{y}) = \{(\mathbf{x_i}, y_i) : i = 1...n\}$;
- $\mathbf{X}$ is a $n \times p$ matrix where $n$ is the number of _samples_ and $p$ is the number of _predictors_ (or _features_);
- $\mathbf{x_i} = (x_{i1},...,x_{in}) \in \mathbf{X}$ is a sample of the training set while $y_i$ is its label;
- $\mathbf{w}^{(t+1)}$ is the $(t+1)$-th update of the weight vector $\mathbf{w}$.

# Iterations' velocity lower bound comparison
> #### Why is it a lower bound for $k=1$
> ![Example](media/img/graph-samples/iter-speed-lb-example.png)

> `a` is the most advanced node (e.g. `a.iteration` $>$ `b.iteration` and `a.iteration` $>$ `c.iteration`), so it is waiting for `b` to finish its calculations, but actually `b` already got the required input from `c`, then, as soon as `b` finishes with the current computation, both `a` and `b` can move to compute to the next step. Then the next iteration will be computed by the most advanced node (it can be either `a` or `b`) after $\frac{1}{\lambda} + \frac{1}{2 \lambda}$ and not after $\frac{1}{\lambda} + \frac{1}{\lambda}$ as considered by the lower bound.

> _(from 2nd-conf-call-report)_

## Actually it is not a lower bound
**Why?**
When `a` ends its calculation for iteration $t$, then it have to wait for `b` to finish iteration $t$ to proceed to compute iteration $t+1$. But there's no guarantee that by the time `a` finishes, then `b` has already got the necessary outcomes from its parents to perform iteration $t$, perhaps `b` is still waiting for `c` to finish iteration $t-1$, while `a` has already finished iteration $t$. Hence $V_l$ is not a lower bound. However it seems to behaves as a lower bound is supposed to do, so for the tests in this report I will exploit it as lower bound.

The speed almost-lower bound is

$$V_l = \frac{\lambda}{1+\sum_{i=1}^{k} \frac{1}{i}}.$$

The bound of the iterations' amount given by $V_l$ is

$$\#it = V_l * time$$

so we're dealing with a line $y = V_l x$, where the amount of iterations is the dependent variable, the time is the independent variable and $V_l$ is the line slope.

For a rate $\lambda$, the slope depends only on the graph's degree $k$. For instance, for $\lambda = 1$:

| Graph | degree $k$ | $V_l$
| :--- |:---: | ---:
| Clique |  $9$  | $\sim 0.34$
| Root and diameter expanders | $2$  | $0.4$
| Cycle |  $1$  | $0.5$
| Diagonal* | $0$  | $1$

_(*) NB: $V_l$ for diagonal graph is not reliable._

## Lower bound for $k=1$

$$\frac{\lambda}{1+\sum_{l=0}^{\delta-1} \sum_{i=1}^{\delta+1-l} \frac{1}{i}}$$

where $\delta = n-1$ is the graph's diameter.

Let $a$ be the most advanced node in computation and suppose it being at iteration $t$. _Remember "node $v$ is at iteration $t$" means that $v$ has already performed the $t$-th update of its local model, so it owns $\mathbf{w}^{(t)}_v$_.

Let $d(u,v)$ be the "directed" distance from $u$ to $v$, e.g. the minimum amount of directed edges one needs to cross in order to go from $u$ to $v$ (obviously edges can be crossed just in the proper direction), then:

- nodes at distance 1 from $a$ must be at least at iteration $t-1$;
- nodes at distance 2 from $a$ must be at least at iteration $t-2$;
- in general, nodes at distance $l : l \leq \delta$ from $a$ must be at least at iteration $t-l$.

Due to exponential memorylessness property, all exponential variables that rule the duration of each node's iteration can be regenerated in any moment. Therefore is legit assuming that right now node $a$ is starting iteration $t$ and nodes at distance $l$ from $a$ are starting iteration $t-l$ for all $l \leq \delta$.

The worst case for node $a$ is the following:
- by the time node $a$ finishes iteration $t$ it doesn't own the required input from its parent $b$ to start iteration $t+1$. So: node $a$ has finished, all random exponential variables can be regenerated, $a$ have to wait for $b$ to finish;
- but $b$ cannot start iteration $t$ since it is still waiting $c$ for its outcome (e.g. $c$ hasn't performed iteration $t-1$ yet); the same for the node $c$ at distance 3 from $a$: it is waiting its parent ($d$), and so on up to the node at distance $\delta$ from $a$, it is actually a straggler, indeed it is slowing down the whole computation.

In this case, the time taken by the slowest node at distance $\delta$ is distributed as $max \{X_1,...,X_n\}$ (where $X_i \sim Exp(\lambda)$ and $n$ is the total amount of computational units). After waiting this time node at distance $\delta-1$ from $a$ can starting computing, etc. A chain of calculations starts up to the node at distance 1 from $a$, and, after they have finished their computations, then $a$ can go on to the next iteration. The time taken by the slowest node at distance $\delta-1$ from $a$ is distributed as $max \{ X_1,...,X_{(n-1)}\}$. In general, the time taken by the slowest node at distance $l$ from $a$ is distributed as $max \{ X_1,...,X_{\delta+1-l}\}$. So the time $a$ have to wait to start iteration $t+1$ is distributed as 

$$Z = \sum_{l=0}^{\delta-1} max \{X_1,...,X_{\delta+1-l}\}$$

so the expected value is

$$\mathbb{E}[Z] = \frac{1}{\lambda} \sum_{l=0}^{\delta-1} \sum_{i=1}^{\delta+1-l} \frac{1}{i} = \frac{1}{\lambda}(\frac{3}{2}\delta+\sum_{i=1}^{\delta-1} \frac{\delta-i}{\delta+2})$$

and the total time taken by $a$ to perform the $(t+1)$-th iteration is $\mathbb{E}[Z] + \frac{1}{\lambda}$.

Therefore the speed is 

$$\frac{\lambda}{1+\sum_{l=0}^{\delta-1} \sum_{i=1}^{\delta+1-l} \frac{1}{i}}$$

Clearly this situation is very rare, **it can occur only when the distance between the most and least advanced nodes is exactly equal to the diameter**, but it gives the most pessimistic lower bound of the time a node should wait in order to perform one iteration. Actually it gives the greatest slowdown the most advanced node can experience due to a straggler, so the worst and most pessimistic scenario.

**Example**: smallest speed for $\lambda = 1$ and $\delta = 1$.

In [2]:
lamb = 1
d = 10
denom = 1
for l in range(d):
    for i in range(1,d+2-l):
        denom += 1/i
print(lamb / denom)

0.039621961887446204


## In the following tests the _almost_-lower bound will be used (and not the latter).

### Iterations over time Lower Bound plot for _10k samples GD test 003_
![Iter/time LB for 10k samples](media/img/tests/test_003_10ksamples_classic/1_iter-lb_time.png)

You can notice that the bound for the diagonal topology is actually an expected value rather than a lower bound.

## Velocity vs degree test
`test_log/test_004_velVSdeg10ksamples10ktime_classic`.

### Test parameters
- `n = 100` (number of computational units).
- `n_samples = 10000`.
- `n_features = 100`.
- `max_time = 10000`.

### Test topologies
Based on degree:
- degree **1**: cycle;
- degree **2**: diameter expander;
- degree **3**, **4**, **8**, **20**, **50**: `generate_d_regular_graph_by_degree(100, k)`;
- degree **99**: clique.

#### How are generated generic regular graphs with degree $k : 3 \leq k \leq N-2$
By exploiting the following function.
```python
def generate_d_regular_graph_by_degree(N, K):
    if N <= 1 or K <= 0:
        edges = []
    else:
        edges = ["i->i+1"]
        for i in range(K-1):
            edges.append("i->i+{}".format(int((i + 1) * N / K)))

    return generate_d_regular_graph_by_edges(N, edges)
```

**Execution examples.**
The body of the function generates edges for the general node $i$ as a list of strings like in snippets below:

In [3]:
N = 100
K = 10
edges = ["i->i+1"]
for i in range(K-1):
    edges.append("i->i+{}".format(int((i + 1) * N / K)))
print(edges)

['i->i+1', 'i->i+10', 'i->i+20', 'i->i+30', 'i->i+40', 'i->i+50', 'i->i+60', 'i->i+70', 'i->i+80', 'i->i+90']


In [4]:
N = 10
K = 4
edges = ["i->i+1"]
for i in range(K-1):
    edges.append("i->i+{}".format(int((i + 1) * N / K)))
print(edges)

['i->i+1', 'i->i+2', 'i->i+5', 'i->i+7']


Then such edges are used from `generate_d_regular_graph_by_edges(N, edges)` to produce the relative $N \times N$ graph's adjacency matrix.

**Example.**
Let's consider the edge list `['i->i+1', 'i->i+2', 'i->i+5', 'i->i+7']` and $N=10$. 

- node $0$ edges will be: $0 \to 1$, $0 \to 2$, $0 \to 5$, $0 \to 7$;
- node $1$ edges will be: $1 \to 2$, $1 \to 3$, $1 \to 6$, $1 \to 8$;
- ...
- node $5$ edges will be: $5 \to 6$, $5 \to 7$, $5 \to 0$, $5 \to 2$; since the $mod\ N$ operation in applied to each edge during evaluation;
- etc.

**Example.**
Running:
```python
generate_d_regular_graph_by_edges(10, ['i->i+1', 'i->i+2', 'i->i+5', 'i->i+7'])
```

outputs:

```
array([[1., 1., 1., 0., 0., 1., 0., 1., 0., 0.],
       [0., 1., 1., 1., 0., 0., 1., 0., 1., 0.],
       [0., 0., 1., 1., 1., 0., 0., 1., 0., 1.],
       [1., 0., 0., 1., 1., 1., 0., 0., 1., 0.],
       [0., 1., 0., 0., 1., 1., 1., 0., 0., 1.],
       [1., 0., 1., 0., 0., 1., 1., 1., 0., 0.],
       [0., 1., 0., 1., 0., 0., 1., 1., 1., 0.],
       [0., 0., 1., 0., 1., 0., 0., 1., 1., 1.],
       [1., 0., 0., 1., 0., 1., 0., 0., 1., 1.],
       [1., 1., 0., 0., 1., 0., 1., 0., 0., 1.]])
```

### Velocity vs degree test - iterations over time
![Velocity vs degree iter/time plot.](media/img/tests/test_004_velVSdeg10ksamples10ktime_classic/1_iter_time_collage.jpg)

_The same test made six times with different seeds (1st in the top left corner, 2nd in the top right corner, 3rd in bottom left corner and 4th and last in the bottom right corner)._

Fixed a degree $k$ and set $\lambda = 1$, $V_i$ is the speed seen in $i$-th test and $V_l$ is the bound defined previously:

| Degree $k$ | $V_1$ | $V_2$ | $V_3$ | $V_4$ | $V_l$ 
| :---: | :---:   | :---:   | :---:   | :---:   | :---:   | :---:   | :---:
| $1$  | $0.501$ | $0.500$ | $0.502$ | $0.503$ | $0.500$
| $2$  | $0.412$ | $0.417$ | $0.418$ | $0.416$ | $0.400$
| $3$  | $0.364$ | $0.363$ | $0.364$ | $0.362$ | $0.353$
| $4$  | $0.351$ | $0.352$ | $0.351$ | $0.353$ | $0.324$
| $8$  | $0.291$ | $0.291$ | $0.290$ | $0.290$ | $0.269$
| $20$ | $0.248$ | $0.247$ | $0.248$ | $0.247$ | $0.217$
| $50$ | $0.211$ | $0.211$ | $0.213$ | $0.211$ | $0.182$
| $99$ | $0.194$ | $0.193$ | $0.194$ | $0.194$ | $0.162$

### Velocity vs degree test - mse over time
![Velocity vs degree mse/time plot.](media/img/tests/test_004_velVSdeg10ksamples10ktime_classic/3_mse_time.png)

### Velocity vs degree test - rmse over time
![Velocity vs degree rmse/time plot.](media/img/tests/test_004_velVSdeg10ksamples10ktime_classic/3_real-mse_time.png)

# Suggestions from set of tests #003
Set of tests #003 ([2nd conf call report](./2nd-conf-call-report.html)) showed some unexpected outcomes:
1. diagonal topology has shown to outperform other topologies in _MSE/RMSE over iterations_ plots when we used to think having an high degreed would have led single step updates to be more accurate;
2. in _MSE/RMSE over time_ plots, starting from a certain time, all topologies seem converging to a high error value (too far from the noise variance), while diagonal topology seemed the only one continuing to improve.

Below I will analyze in details these two points trying to figure out their meaning and causes.

## Single node model inspection
Diagonal topology performs way too better than expected. Therefore I decided to inspect single node performance, e.g. whether each node is able to reach a good solution alone (due to training set size for instance) or whether the aggregation stage (weight vectors average) leads local bad models to produce a good new one. Thus below I provide a test whose plots take into account local metrics of nodes.

In the following test, MSE (and RMSE) are computed by averaging local errors in nodes:

$$MSE = \frac{1}{n} \sum_{v=1}^{n} MSE_v$$

$$RMSE = \frac{1}{n} \sum_{v=1}^{n} RMSE_v$$

where $MSE_v$ and $RMSE_v$ are respectively the mean squared error and the real mean squared error of node $v$:

$$MSE_v = \frac{1}{|\mathcal{X}_v|} \sum_{i=1}^{|\mathcal{X}_v|} (y_i - \hat{y}_i)^2$$

$$RMSE_v = \frac{1}{|\mathcal{X}_v|} \sum_{i=1}^{|\mathcal{X}_v|} (\tilde{y}_i - \hat{y}_i)^2$$
 
- $\mathcal{X}_v \subset \mathcal{X}$ is the subset of the training set assigned to $v$;
- $y_i$ is the outcome provided by the training set;
- $\hat{y}_i$ is the value computed using the model $\mathbf{w}_v$;
- $\tilde{y}_i$ is the noiseless $y_i$.

These MSE and RMSE will be called "**ALT metrics**" (_alternatives_ metrics).

## 10k samples ALT metrics 500k time GD test
- `n_samples = 10000`.
- `max_time = 500000`: simulation stops when time reaches $500000$s.
- All other parameters are left untouched.

### 10k samples ALT metrics 500k time GD - iter/time
![10k samples ALT metrics GD test iter/time.](media/img/tests/test_004_10ksamplesALTmetrics500ktime_classic/1_iter_time.png)

### 10k samples ALT metrics 500k time GD - mse/iter
![10k samples ALT metrics 500k time GD test mse/iter.](media/img/tests/test_004_10ksamplesALTmetrics500ktime_classic/2_mse_iter.png)

![10k samples ALT metrics 500k time GD test mse/iter zoom.](media/img/tests/test_004_10ksamplesALTmetrics500ktime_classic/2_mse_iter_zoom.png)

### 10k samples ALT metrics 500k time GD - rmse/iter
![10k samples ALT metrics 500k time GD test rmse/iter.](media/img/tests/test_004_10ksamplesALTmetrics500ktime_classic/2_real-mse_iter.png)

![10k samples ALT metrics 500k time GD test rmse/iter zoom.](media/img/tests/test_004_10ksamplesALTmetrics500ktime_classic/2_real-mse_iter_zoom.png)

### 10k samples ALT metrics 500k time GD - mse/time

#### 10k samples ALT metrics 500k time GD - mse/time (log scale)
![10k samples ALT metrics 500k time GD test mse/time.](media/img/tests/test_004_10ksamplesALTmetrics500ktime_classic/3_mse_time.png)

![10k samples ALT metrics 500k time GD test mse/time zoom 1.](media/img/tests/test_004_10ksamplesALTmetrics500ktime_classic/3_mse_time_zoom_1.png)

#### 10k samples ALT metrics 500k time GD - mse/time (linear scale zoom)
![10k samples ALT metrics 500k time GD test mse/time zoom 2.](media/img/tests/test_004_10ksamplesALTmetrics500ktime_classic/3_mse_time_zoom_2.png)

![10k samples ALT metrics 500k time GD test mse/time zoom 3.](media/img/tests/test_004_10ksamplesALTmetrics500ktime_classic/3_mse_time_zoom_3.png)

### 10k samples ALT metrics 500k time GD - rmse/time
![10k samples ALT metrics 500k time GD test rmse/time.](media/img/tests/test_004_10ksamplesALTmetrics500ktime_classic/3_real-mse_time.png)

![10k samples ALT metrics 500k time GD test rmse/time zoom 1.](media/img/tests/test_004_10ksamplesALTmetrics500ktime_classic/3_real-mse_time_zoom_1.png)

![10k samples ALT metrics 500k time GD test rmse/time zoom 2.](media/img/tests/test_004_10ksamplesALTmetrics500ktime_classic/3_real-mse_time_zoom_2.png)

![10k samples ALT metrics 500k time GD test rmse/time zoom 3.](media/img/tests/test_004_10ksamplesALTmetrics500ktime_classic/3_real-mse_time_zoom_3.png)

### What is happening?
$\mathcal{X}_v$ are taken from the same training set which is generated exploiting one single function, so a model that well behaves on a subset of samples $\mathcal{X}_a$ should not misbehave on any other subset $\mathcal{X}_i$. Well, this is a strong generalization, there are many conditions that should be satisfied (for instance $|\mathcal{X}_i| \geq p$, e.g. the amount of samples in each training set subset have to be at least equal to the number of predictors), but we can assume it as true. Hence, averaging two (ore more) weight vectors computed on different subsets, but belonging to the same original training set $\mathcal{X}$, should fasten the convergence. The average of multiple weight vectors should produce an aggregate vector which behaves overall better then them, in particular after the aggregation stage has been performed several times. The only topology unable to average (communicate) is the diagonal one. That's why, for it, local models don't improve in time such as the ones in other topologies.

The following conclusions can be made:
- a single node alone (in diagonal topology), without communicating with any other, doesn't achieve a good enough model with respect to other topologies;
- the goodness of the model computed by the diagonal topology in our past tests is a consequence of the aggregation stage, moreover diagonal prediction model has turned out to improve always faster than the ones achieved by other topologies with communication enabled; why it always happens is not clear at the moment, this may require some further inspections. It seems that making the average just once at the end of the whole computation affects the goodness of the model as much as making the average after each weights' update. The iteration speed boost due to absence of dependencies ranges from x2 (compared with cycle) to about x6 (compared with clique), this enables the diagonal topology being so efficient.

All this statements must be reviewed after making a lot of other tests especially with other kind of datasets.

## Models long term convergence
In the tests below the time is $50$x the one used in past tests, in order to see how topologies behave when approaching the minimum.

## 10k samples 500k time GD test
Here MSE and RMSE are computed in the regular way (and not averaging local errors).

Simulation stops when time reaches $500'000$s.

This simulation has taken $3h14m26s$. I tried the same kind of test with `n_samples = 100000` instead of $10000$, it took $11h38m19s$, however it won't be reported since the results are almost the same.

### 10k samples 500k time GD - iter/time
![10k samples 500k time GD test iter/time.](media/img/tests/test_004_10ksamples500ktime_classic/1_iter_time.png)

### 10k samples 500k time GD - mse/iter
![10k samples 500k time GD test mse/iter.](media/img/tests/test_004_10ksamples500ktime_classic/2_mse_iter.png)

![10k samples 500k time GD test mse/iter zoom.](media/img/tests/test_004_10ksamples500ktime_classic/2_mse_iter_zoom.png)

### 10k samples 500k time GD - rmse/iter
![10k samples 500k time GD test rmse/iter.](media/img/tests/test_004_10ksamples500ktime_classic/2_real-mse_iter.png)

![10k samples 500k time GD test rmse/iter zoom.](media/img/tests/test_004_10ksamples500ktime_classic/2_real-mse_iter_zoom.png)

### 10k samples 500k time GD - mse/time
![10k samples 500k time GD test mse/iter.](media/img/tests/test_004_10ksamples500ktime_classic/3_mse_time.png)

![10k samples 500k time GD test mse/iter zoom.](media/img/tests/test_004_10ksamples500ktime_classic/3_mse_time_zoom_1.png)

![10k samples 500k time GD test mse/iter zoom.](media/img/tests/test_004_10ksamples500ktime_classic/3_mse_time_zoom_2.png)

### 10k samples 500k time GD - rmse/time
![10k samples 500k time GD test rmse/iter.](media/img/tests/test_004_10ksamples500ktime_classic/3_real-mse_time.png)

![10k samples 500k time GD test rmse/iter zoom.](media/img/tests/test_004_10ksamples500ktime_classic/3_real-mse_time_zoom_1.png)

![10k samples 500k time GD test rmse/iter zoom.](media/img/tests/test_004_10ksamples500ktime_classic/3_real-mse_time_zoom_2.png)

Actually what seemed from short-time tests was not true, all topologies continue improving while time goes on, however the diagonal outperforms the others also near the minimum. As the time goes to infinite all topologies will achieve a prediction error equal to $1$ (due to noise standard deviation being equal to $1$).